In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import numpy as np
import math
import keras
import tensorflow as tf
from keras.layers import Activation, Lambda, Dense,BatchNormalization, Dropout, LSTM,  Conv2D, Conv1D, Flatten, MaxPooling2D, AveragePooling2D, Input, ZeroPadding2D, Reshape , Add
from keras.models import Model
import keras.backend as backend
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
from scipy import misc
from sklearn import preprocessing
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from keras import optimizers
import pylab as plt
import matplotlib.pyplot as plt
from pyts.image import RecurrencePlot
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, base,Trials
import csv
dfz = pd.read_excel('newdata.xlsx')
df=dfz[:5665]
df=df.drop('Date (GMT)',axis=1)
df=df.fillna(method='ffill')
setz=[0,1,2,3,4,5,7,9,11,12,13,15,17,25,26,27,28,30,32,48,49,50,
      51,52,53,54,55,56,59,60,61,62,63,65,67,70,72,74,77,80,83,85,88,91,166,167,168,170,172] 
df=df.iloc[:,setz]
df=df.dropna(axis=1,how='all')


def Preprocessin(df2, setz ,windowlength=30, prediction_length=5, perz=0.2, threshz='distance'):
    dftrain=df.copy()[300:4000]
    dfvalid=df.copy()[4000:5220]
    dftest = df.copy()[5220:]

    sclr = preprocessing.StandardScaler()
    sclr = sclr.fit(dftrain)

    trainscaled = sclr.transform(dftrain)
    valscaled = sclr.transform(dfvalid)
    testscaled = sclr.transform(dftest)

    trainarray=trainscaled[:,3].copy()                                   #extract array of particular feature  
    validarray=valscaled[:,3].copy()                                    
    testarray=testscaled[:,3].copy()              #,tmean,tvar=tanz(arrayzz[:,3].copy(), True)   
    print(trainarray.shape)         

    pagestrain = trainarray.shape[0] - windowlength - prediction_length
    pagesvalid = validarray.shape[0]  - windowlength - prediction_length
    pagestest = testarray.shape[0] - windowlength - prediction_length-2

    OUTPUT_train = np.asarray([[ trainarray[i+k+windowlength] for i in range(prediction_length)]  for k in range( pagestrain)])
    OUTPUT_val = np.asarray([[validarray[i+k+windowlength] for i in range(prediction_length)] for k in range(pagesvalid)])
    OUTPUT_test = np.asarray([[ testarray[i+k+windowlength] for i in range(prediction_length )] for k in range(pagestest)])

    for feature in range(np.array(dftrain).shape[1]):
        print(feature)
        trainarray=trainscaled[:,feature].copy()                                   #extract array of particular feature  
        validarray=valscaled[:,feature].copy()                                    
        testarray=testscaled[:,feature].copy()                 
        #save 2d time series data as 3d list with batch_size=windowlength
        trainarray = np.array([[[ trainarray[i+k] for i in  range(windowlength)] for t in range(1)] for k in range(pagestrain)])
        validarray = np.array([[[validarray[i+k] for i in range(windowlength)] for t in range(1)] for k in range(pagesvalid)]) 
        testarray = np.array([[[ testarray[i+k] for i in range(windowlength)]for t in range(1)] for k in range(pagestest)])
        if feature == 0:
            INPUT_train_lstm = trainarray.copy()
            INPUT_valid_lstm = validarray.copy()
            INPUT_test_lstm = testarray.copy()
        else:
            INPUT_train_lstm = np.append(INPUT_train_lstm,trainarray,axis=1)
            INPUT_valid_lstm = np.append(INPUT_valid_lstm,validarray,axis=1)
            INPUT_test_lstm=np.append(INPUT_test_lstm,testarray,axis=1)

    INPUT_train_lstm=INPUT_train_lstm.swapaxes(1,2)
    INPUT_valid_lstm=INPUT_valid_lstm.swapaxes(1,2)

    return  INPUT_train_lstm, INPUT_test_lstm,INPUT_valid_lstm,OUTPUT_val, np.array(OUTPUT_train), np.array(OUTPUT_test) #, tmean, tvar


input_train,input_test,input_valid,out_valid,out_train,out_test = Preprocessin(df,setz)

train_data = tf.data.Dataset.from_tensor_slices((input_train,out_train))
train_data = train_data.cache().shuffle(10000).batch(56).repeat()
val_data = tf.data.Dataset.from_tensor_slices((input_valid,out_valid))
val_data = val_data.shuffle(10000).batch(56).repeat()


In [ ]:
best

In [ ]:
def modelcall(dictz): 
    f_c1=int(dictz['f_c1'])
    f_c2=int(dictz['f_c2'])
    f_ls=int(dictz['f_ls'])
    c1kernel=int(dictz['c1kernel'])
    c2kernel=int(dictz['c2kernel'])
    c1stride=int(dictz['c1stride'])
    epochz=int(dictz['epochz'])
    learningrate = int(dictz['learningrate'])
    pooling = int(dictz['pooling'])  
    dropz = int(dictz['dropz'])  
    inp = tf.keras.layers.Input(shape=(30,49))
    x = tf.keras.layers.Conv1D(f_c1,kernel_size=3,activation='relu')(inp)
    x = tf.keras.layers.Conv1D(f_c2,kernel_size=2,activation='relu')(x)
    if pooling:
        x = tf.keAddras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.LSTM(f_ls,return_sequences=True,activation='relu')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(dropz)(x)
    out = tf.keras.layers.Dense(5)(x)
    model=tf.keras.Model(inp,out)
    stepsperepoch=np.floor(input_train.shape[0]/56)
    stepsperval=np.floor(input_valid.shape[0]/56)
    rms = tf.keras.optimizers.RMSprop(learning_rate=learningrate,decay=0.01,momentum=0.85)
    model=tf.keras.Model(inp,out)
    model.compile(optimizer=rms,loss='mean_squared_error')
    hist=model.fit(train_data,validation_data=val_data,shuffle=False,epochs=epochz,steps_per_epoch=stepsperepoch,validation_steps=stepsperval)
    return {
        'loss': min(hist.history['loss']),
        'val_loss': min(hist.history['val_loss']),
        'status': STATUS_OK,
          }
  
spacez=hp.choice('a',[
      {  
    'f_c1' : hp.uniform('f_c1', 64,128 ),
    'f_c2' : hp.uniform('f_c2', 128,256 ),
    'f_ls' : hp.uniform('f_ls', 32,128 ),      
    'c1kernel' : hp.choice('c1kernel', [2,3,4] ),
    'c2kernel' : hp.choice('c2kernel', [2,3,4] ),
    'c1stride' : hp.choice('c1stride', [1,2] ),
    'epochz' : hp.uniform('epochz', 100,3000),
    'learningrate' : hp.uniform('learningrate', 0.005,0.1),
    'pooling' : hp.choice('pooling', [True,False] ),
    'dropz' : hp.uniform('dropz', 0,0.4 )
      }])
trials=Trials()
best = fmin(modelcall,
            space=spacez,
            algo=tpe.suggest,
            trials=trials,
            max_evals=25)
w = csv.writer(open("tuning111111111.csv", "w"))
for key, val in best.items():
    w.writerow([key, val])


In [ ]:
spacez